In [1]:
!git clone https://github.com/mateoKutnjak/pose_estimation.git

fatal: destination path 'pose_estimation' already exists and is not an empty directory.


In [0]:
import sys
sys.path.append('/content/pose_estimation')

In [0]:
from google.colab import drive
import models

In [7]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
_IMAGES_DIR = '/content/drive/My Drive/_images'
_ANNOTATIONS_JSON_FILE = '/content/drive/My Drive/_annotations/annotations.json'
IMAGES_DIR = '/content/drive/My Drive/images'
ANNOTATIONS_JSON_FILE = '/content/drive/My Drive/annotations/annotations.json'
LOG_DIR = '/content/drive/My Drive/logs'
INPUT_SHAPE = (256, 256, 3)
OUTPUT_SHAPE = (64, 64, 16)
BATCH_SIZE = 8
STACKS_NUM = 4
EPOCHS = 10

CHECKPOINT_EPOCH = None

In [0]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 1.0

k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
hg_model = HourglassModel(
        images_dir=_IMAGES_DIR,
        annotations_json_file=_ANNOTATIONS_JSON_FILE,
        log_dir=LOG_DIR,
        batch_size=BATCH_SIZE,
        stacks_num=STACKS_NUM,
        input_shape=INPUT_SHAPE,
        output_shape=OUTPUT_SHAPE,
    )

In [13]:
if CHECKPOINT_EPOCH is None:
    hg_model.build()
    hg_model.train(epochs=EPOCHS)
else:
    hg_model.resume_train(EPOCHS, CHECKPOINT_EPOCH)

Instructions for updating:
Colocations handled automatically by placer.
